importaciones

In [1]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import json
import ast
import re
from textblob import TextBlob

importacion de la data y primeros vistazos

In [2]:
ruta_review = '../archivos_crudo/australian_user_reviews.json'

# Se lee de cada línea del dataset
filas_review = []
with open(ruta_review) as f:
    for line in f.readlines():
        filas_review.append(ast.literal_eval(line))

# Se convierte en dataframe
df_reviews = pd.DataFrame(filas_review)
df_reviews.to_parquet

<bound method DataFrame.to_parquet of                  user_id                                           user_url  \
0      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
1                js41637               http://steamcommunity.com/id/js41637   
2              evcentric             http://steamcommunity.com/id/evcentric   
3                  doctr                 http://steamcommunity.com/id/doctr   
4              maplemage             http://steamcommunity.com/id/maplemage   
...                  ...                                                ...   
25794  76561198306599751  http://steamcommunity.com/profiles/76561198306...   
25795           Ghoustik              http://steamcommunity.com/id/Ghoustik   
25796  76561198310819422  http://steamcommunity.com/profiles/76561198310...   
25797  76561198312638244  http://steamcommunity.com/profiles/76561198312...   
25798        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   

             

In [3]:
# le echo un primer vistazo a la data 
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [4]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [5]:
df_reviews.isnull().sum()

user_id     0
user_url    0
reviews     0
dtype: int64

empiezo eliminando los id duplicados 

In [6]:
# verifico si hay o no duplicados por cada 'user_id' y cuantos
print(df_reviews['user_id'].duplicated().sum())


314


In [7]:
# creo un dataframe con los duplicados
duplicate_ids = df_reviews['user_id'].duplicated()
df_reviews[duplicate_ids]

,user_id,user_url,reviews
456,bokkkbokkk,http://steamcommunity.com/id/bokkkbokkk,"[{'funny': '', 'posted': 'Posted September 24,..."
1182,ImSeriouss,http://steamcommunity.com/id/ImSeriouss,"[{'funny': '', 'posted': 'Posted January 10, 2..."
1456,76561198062039159,http://steamcommunity.com/profiles/76561198062...,"[{'funny': '', 'posted': 'Posted August 24, 20..."
1477,76561198045009232,http://steamcommunity.com/profiles/76561198045...,"[{'funny': '', 'posted': 'Posted October 31, 2..."
1746,nitr0ticwolf,http://steamcommunity.com/id/nitr0ticwolf,"[{'funny': '', 'posted': 'Posted December 12, ..."
...,...,...,...
17819,76561198076474887,http://steamcommunity.com/profiles/76561198076...,"[{'funny': '', 'posted': 'Posted April 12.', '..."
17916,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
18028,76561198075591109,http://steamcommunity.com/profiles/76561198075...,"[{'funny': '', 'posted': 'Posted December 26, ..."
18234,76561198092022514,http://steamcommunity.com/profiles/76561198092...,"[{'funny': '', 'posted': 'Posted July 3.', 'la..."


In [8]:
# elimino los id´s duplicados
df_reviews = df_reviews[~duplicate_ids] 
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


empiezo con el desanidado de la columna reviews y su normalizacion

In [9]:
# Se transforma a columnas cada elemento de las listas
df_reviews2 = pd.json_normalize(df_reviews['reviews'])
df_reviews2.head()

,0,1,2,3,4,5,6,7,8,9
0,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None


In [10]:
# Se agrega el 'user_id' y 'user_url' a las columnas separadas 
df_reviews2 = pd.concat([df_reviews[['user_id', 'user_url']], df_reviews2], axis=1)
df_reviews2

,user_id,user_url,0,1,2,3,4,5,6,7,8,9
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,evcentric,http://steamcommunity.com/id/evcentric,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,doctr,http://steamcommunity.com/id/doctr,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,maplemage,http://steamcommunity.com/id/maplemage,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
17819,NaN,NaN,"{'funny': '', 'posted': 'Posted July 12, 2014....",None,None,None,None,None,None,None,None,None
17916,NaN,NaN,"{'funny': '', 'posted': 'Posted December 27, 2...","{'funny': '', 'posted': 'Posted September 2, 2...","{'funny': '', 'posted': 'Posted August 23, 201...",None,None,None,None,None,None,None
18028,NaN,NaN,"{'funny': '', 'posted': 'Posted September 22, ...",None,None,None,None,None,None,None,None,None
18234,NaN,NaN,"{'funny': '', 'posted': 'Posted July 7, 2014.'...",None,None,None,None,None,None,None,None,None


In [11]:
# Se utiliza pd.melt para transformar las columnas en filas conservando el 'user_id' y 'user_url'
df_reviews2 = pd.melt(df_reviews2, id_vars=['user_id', 'user_url'], 
                       value_vars=list(range(9)),
                       value_name='reviews')
df_reviews2

,user_id,user_url,variable,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,js41637,http://steamcommunity.com/id/js41637,0,"{'funny': '', 'posted': 'Posted June 24, 2014...."
2,evcentric,http://steamcommunity.com/id/evcentric,0,"{'funny': '', 'posted': 'Posted February 3.', ..."
3,doctr,http://steamcommunity.com/id/doctr,0,"{'funny': '', 'posted': 'Posted October 14, 20..."
4,maplemage,http://steamcommunity.com/id/maplemage,0,"{'funny': '3 people found this review funny', ..."
...,...,...,...,...
232186,NaN,NaN,8,None
232187,NaN,NaN,8,None
232188,NaN,NaN,8,None
232189,NaN,NaN,8,None


In [12]:
# Se eliminan las filas con valor None
df_reviews2 = df_reviews2.dropna()

In [13]:
# Se separan por columnas cada una de las claves de 'reviews'
df_reviews = df_reviews2['reviews'].apply(pd.Series, dtype='object')
df_reviews = df_reviews.add_prefix('reviews_')
df_reviews.head()

,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,,"Posted October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,3 people found this review funny,"Posted April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud


In [14]:
# Se une con el 'user_id' y 'user_url' 
df_reviews = pd.concat([df_reviews2[['user_id', 'user_url']], df_reviews], axis=1)
df_reviews.head()

,user_id,user_url,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,,"Posted October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,"Posted April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud


In [15]:
df_reviews

,user_id,user_url,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,,"Posted October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,"Posted April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud
...,...,...,...,...,...,...,...,...,...
231291,How51,http://steamcommunity.com/id/How51,,"Posted August 15, 2014.","Last edited November 3, 2014.",440,No ratings yet,True,TF2 is alot of fun and its really good but the...
231293,76561198111410893,http://steamcommunity.com/profiles/76561198111...,,"Posted August 2, 2014.",,304930,No ratings yet,True,Fun game with friends
231419,zaza147,http://steamcommunity.com/id/zaza147,,"Posted July 31, 2015.",,265630,No ratings yet,True,So Fun!! :D
231499,lifeonhigh,http://steamcommunity.com/id/lifeonhigh,,"Posted December 20, 2015.",,304050,No ratings yet,True,"This game is great. The only thing is,Why cant..."


In [16]:
df_reviews['reviews_last_edited'].isnull().sum()

0

In [17]:
df_reviews['reviews_posted'].isnull().sum()

0

In [18]:
# Se eliminan las columnas 'reviews_funny' y 'reviews_last_edited' 
#ya que considero que ya no son de utilidad
df_reviews = df_reviews.drop(columns=['reviews_funny', 'reviews_last_edited'])
df_reviews

,user_id,user_url,reviews_posted,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,Posted February 3.,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,"Posted October 14, 2013.",250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,"Posted April 15, 2014.",211420,35 of 43 people (81%) found this review helpful,True,Git gud
...,...,...,...,...,...,...,...
231291,How51,http://steamcommunity.com/id/How51,"Posted August 15, 2014.",440,No ratings yet,True,TF2 is alot of fun and its really good but the...
231293,76561198111410893,http://steamcommunity.com/profiles/76561198111...,"Posted August 2, 2014.",304930,No ratings yet,True,Fun game with friends
231419,zaza147,http://steamcommunity.com/id/zaza147,"Posted July 31, 2015.",265630,No ratings yet,True,So Fun!! :D
231499,lifeonhigh,http://steamcommunity.com/id/lifeonhigh,"Posted December 20, 2015.",304050,No ratings yet,True,"This game is great. The only thing is,Why cant..."


In [19]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57397 entries, 0 to 231501
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   user_id            57397 non-null  object
 1   user_url           57397 non-null  object
 2   reviews_posted     57397 non-null  object
 3   reviews_item_id    57397 non-null  object
 4   reviews_helpful    57397 non-null  object
 5   reviews_recommend  57397 non-null  bool  
 6   reviews_review     57397 non-null  object
dtypes: bool(1), object(6)
memory usage: 3.1+ MB


In [20]:
df_reviews.isnull().sum()

user_id              0
user_url             0
reviews_posted       0
reviews_item_id      0
reviews_helpful      0
reviews_recommend    0
reviews_review       0
dtype: int64

yo considero que la data ya fue tratada y limpiada. por lo que finalizo el Etl del mismo y lo guardo en archivos limpios


In [21]:
df_reviews.to_parquet('reviews.parquet')